In [262]:
import numpy as np
import sympy as sp

In [263]:
# Psi (function)
psi, rs, w = sp.symbols("psi, r_*, omega")
# Real and positive values
r, m, l, y = sp.symbols("r, m, ell, y", positive = True)

### Schwarzschild Metric

$$
ds^2 = \left(1 - \frac{2m}{r^2}\right)\,dt^2 + \left(1 - \frac{2m}{r^2}\right)^{-1}\,dr^2 + r^2\,d\Omega_2^2 
$$

Define $G(r)$, $F(r)$, and $G(r)$

In [264]:
G = (1 - (2*m) / r)
G

-2*m/r + 1

In [265]:
F = 1 / G
F

1/(-2*m/r + 1)

In [266]:
H = r**2
H

r**2

Calculate derivatives $G'(r)$, $F'(r)$, $H'(r)$, $H''(r)$

In [267]:
Hp = sp.Derivative(H, r, evaluate = True)
Hp

2*r

In [268]:
Hpp = sp.Derivative(Hp, r, evaluate = True)
Hpp

2

In [269]:
Gp = sp.Derivative(G, r, evaluate = True)
Gp

2*m/r**2

In [270]:
Fp = sp.Derivative(F, r, evaluate = True)
Fp

-2*m/(r**2*(-2*m/r + 1)**2)

Compute radial function $R(r)$

In [271]:
R = 2/H + 1/F * ( Gp * Hp / (4 * G * H) - Fp * Hp / (4 * F * H) - 3 * Hp**2 / (4 * H**2) + Hpp/(2 * H) )
R

(-2*m/r + 1)*(m/(r**3*(-2*m/r + 1)) + 4*m/(r**3*(-8*m/r + 4)) - 2/r**2) + 2/r**2

In [272]:
R.simplify()

6*m/r**3

Compute potential $V(r)$

In [273]:
V = G * (l*(l+1)/H - R.simplify())
V

(-2*m/r + 1)*(ell*(ell + 1)/r**2 - 6*m/r**3)

Differential equation

In [274]:
# Define second-order derivative
ddpsiddrstar = sp.Derivative( sp.Derivative(psi, rs, evaluate = False), rs, evaluate = False )
ddpsiddrstar

Derivative(psi, (r_*, 2))

In [275]:
eq = ddpsiddrstar + (w**2 - V)*psi
eq

psi*(omega**2 - (-2*m/r + 1)*(ell*(ell + 1)/r**2 - 6*m/r**3)) + Derivative(psi, (r_*, 2))

Calculate tortoise coordinate
$$
dr_* = \sqrt{\frac{F(r)}{G(r)}}dr
$$

under change of coordinates:
$$
y = 1-\frac{2m}{r} \rightarrow r = \frac{2m}{1-y}
$$

Expand second derivative over tortoise coordinate as chain rule of derivatives
$$
\frac{d^2\psi(r)}{dr_*^2} = \left(\frac{d^2y}{dr_*^2}\right)\frac{d\psi(y)}{dy} + \left(\frac{dy}{dr_*}\right)^2\frac{d^2\psi}{dy^2}
$$

where:
$$
\frac{d^2y}{dr_*^2} = - \frac{d/dy(dr_*/dy)}{(dr_*/dy)^3}
$$

Define change of variables

In [276]:
# Define change of variable y = ...
var_change_y = 1 - 2*m/r
var_change_y

-2*m/r + 1

In [277]:
# Define change of variable r = ...
var_change_r = 2*m/(1-y)
var_change_r

2*m/(1 - y)

In [278]:
# Compute dr = ... dy
drdy = sp.Derivative(var_change_r, y, evaluate = True)
drdy

2*m/(1 - y)**2

In [279]:
# Compute dr_*/dy
drstardr = (sp.sqrt(F / G)).subs(r, var_change_r) * drdy
drstardr

2*m/(y*(1 - y)**2)

In [280]:
# Compute dy/dr_*
dydrstar = 1 / drstardr
dydrstar

y*(1 - y)**2/(2*m)

In [281]:
# Compute d^2y/dr_*^2
ddyddrstar = - (sp.Derivative(drstardr, y, evaluate = True) / (drstardr)**3).simplify()
ddyddrstar

y*(y - 1)**3*(3*y - 1)/(4*m**2)

In [282]:
# Define new psi(y)
psi_y = sp.Function('psi')(y)
psi_y

psi(y)

In [298]:
# Compute second-order derivative of psi in terms of y
ddpsiddy = ddyddrstar * sp.Derivative(psi_y, y, evaluate = False) + dydrstar**2 * sp.Derivative(sp.Derivative(psi_y, y, evaluate = False), y, evaluate = False)
ddpsiddy

y**2*(1 - y)**4*Derivative(psi(y), (y, 2))/(4*m**2) + y*(y - 1)**3*(3*y - 1)*Derivative(psi(y), y)/(4*m**2)

In [299]:
# First order derivative
dpsidy = sp.Derivative(psi_y, y, evaluate = False)
dpsidy

Derivative(psi(y), y)

Substitute $y$ variable into equation

In [300]:
eq_y = eq.subs(r, var_change_r)
eq_y

psi*(omega**2 - y*(ell*(1 - y)**2*(ell + 1)/(4*m**2) - 3*(1 - y)**3/(4*m**2))) + Derivative(psi, (r_*, 2))

Substitute $d^2/dr_*^2$ into equation

In [301]:
eq_yy = eq_y.subs( ddpsiddrstar, ddpsiddy )
eq_yy

psi*(omega**2 - y*(ell*(1 - y)**2*(ell + 1)/(4*m**2) - 3*(1 - y)**3/(4*m**2))) + y**2*(1 - y)**4*Derivative(psi(y), (y, 2))/(4*m**2) + y*(y - 1)**3*(3*y - 1)*Derivative(psi(y), y)/(4*m**2)

Normalize second-order derivative term

In [302]:
ddpsiddy_coeff = eq_yy.coeff(sp.Derivative(sp.Derivative(psi_y, y), y))
ddpsiddy_coeff

y**2*(1 - y)**4/(4*m**2)

In [303]:
dpsidy_coeff = eq_yy.coeff(sp.Derivative(psi_y, y))
dpsidy_coeff

y*(y - 1)**3*(3*y - 1)/(4*m**2)

In [304]:
psi_coeff = eq_yy.coeff(psi)
psi_coeff

omega**2 - y*(ell*(1 - y)**2*(ell + 1)/(4*m**2) - 3*(1 - y)**3/(4*m**2))

In [305]:
# New normalized coefficient for first derivative term
dpsidy_coeff_norm = (dpsidy_coeff / ddpsiddy_coeff).simplify()
dpsidy_coeff_norm

(3*y - 1)/(y*(y - 1))

In [306]:
# New normalzied coefficient for psi term
psi_coeff_norm = (psi_coeff / ddpsiddy_coeff).simplify()
psi_coeff_norm

(4*m**2*omega**2 - y*(y - 1)**2*(ell*(ell + 1) + 3*y - 3))/(y**2*(y - 1)**4)

In [307]:
# Generate new normalized equation
eq_yy_norm = sp.Derivative(sp.Derivative(psi_y, y), y) + dpsidy * dpsidy_coeff_norm + psi_y * psi_coeff_norm
eq_yy_norm

Derivative(psi(y), (y, 2)) + (3*y - 1)*Derivative(psi(y), y)/(y*(y - 1)) + (4*m**2*omega**2 - y*(y - 1)**2*(ell*(ell + 1) + 3*y - 3))*psi(y)/(y**2*(y - 1)**4)

Define boundary conditions
$$
\psi(y) \sim y^{-2i\omega m}(1-y)^{-2i\omega m}\exp\left(\frac{2i\omega m}{1-y}\right)\phi(y)
$$

In [308]:
# Define new function
phi_y = sp.Function('phi')(y)
phi_y

phi(y)

In [310]:
# Define new psi from boundary conditions
psi_y_bc = y**(-2*sp.I*w*m)*(1-y)**(-2*sp.I*w*m)*sp.exp(2*sp.I*w*m/(1-y))*phi_y
psi_y_bc

phi(y)*exp(2*I*m*omega/(1 - y))/(y**(2*I*m*omega)*(1 - y)**(2*I*m*omega))

In [ ]:
# Compute first derivative
dpsi_y_bc_dy = sp.Derivative(psi_y_bc, y, evaluate = True)
dpsi_y_bc_dy

2*I*m*omega*phi(y)*exp(2*I*m*omega/(1 - y))/(y**(2*I*m*omega)*(1 - y)*(1 - y)**(2*I*m*omega)) + 2*I*m*omega*phi(y)*exp(2*I*m*omega/(1 - y))/(y**(2*I*m*omega)*(1 - y)**2*(1 - y)**(2*I*m*omega)) - 2*I*m*omega*phi(y)*exp(2*I*m*omega/(1 - y))/(y*y**(2*I*m*omega)*(1 - y)**(2*I*m*omega)) + exp(2*I*m*omega/(1 - y))*Derivative(phi(y), y)/(y**(2*I*m*omega)*(1 - y)**(2*I*m*omega))

In [317]:
# Compute second derivative
ddpsi_y_bc_ddy = sp.Derivative(dpsi_y_bc_dy, y, evaluate = True)
ddpsi_y_bc_ddy

-4*m**2*omega**2*phi(y)*exp(2*I*m*omega/(1 - y))/(y**(2*I*m*omega)*(1 - y)**2*(1 - y)**(2*I*m*omega)) - 8*m**2*omega**2*phi(y)*exp(2*I*m*omega/(1 - y))/(y**(2*I*m*omega)*(1 - y)**3*(1 - y)**(2*I*m*omega)) - 4*m**2*omega**2*phi(y)*exp(2*I*m*omega/(1 - y))/(y**(2*I*m*omega)*(1 - y)**4*(1 - y)**(2*I*m*omega)) + 8*m**2*omega**2*phi(y)*exp(2*I*m*omega/(1 - y))/(y*y**(2*I*m*omega)*(1 - y)*(1 - y)**(2*I*m*omega)) + 8*m**2*omega**2*phi(y)*exp(2*I*m*omega/(1 - y))/(y*y**(2*I*m*omega)*(1 - y)**2*(1 - y)**(2*I*m*omega)) - 4*m**2*omega**2*phi(y)*exp(2*I*m*omega/(1 - y))/(y**2*y**(2*I*m*omega)*(1 - y)**(2*I*m*omega)) + 4*I*m*omega*exp(2*I*m*omega/(1 - y))*Derivative(phi(y), y)/(y**(2*I*m*omega)*(1 - y)*(1 - y)**(2*I*m*omega)) + 2*I*m*omega*phi(y)*exp(2*I*m*omega/(1 - y))/(y**(2*I*m*omega)*(1 - y)**2*(1 - y)**(2*I*m*omega)) + 4*I*m*omega*exp(2*I*m*omega/(1 - y))*Derivative(phi(y), y)/(y**(2*I*m*omega)*(1 - y)**2*(1 - y)**(2*I*m*omega)) + 4*I*m*omega*phi(y)*exp(2*I*m*omega/(1 - y))/(y**(2*I*m*omega)*

In [324]:
# Substitute into the equations
eq_yy_norm_bc = eq_yy_norm.subs( psi_y, psi_y_bc ).doit()
eq_yy_norm_bc

(8*m**2*omega**2*phi(y)/(y - 1)**3 - 8*m**2*omega**2*phi(y)/(y*(y - 1)) + 8*m**2*omega**2*phi(y)/(y*(y - 1)**2) - 4*I*m*omega*Derivative(phi(y), y)/(y - 1) - 2*m*omega*(2*m*omega - I)*phi(y)/(y - 1)**2 + 4*I*m*omega*Derivative(phi(y), y)/(y - 1)**2 - 4*m*omega*(m*omega/(y - 1) + I)*phi(y)/(y - 1)**3 - 4*I*m*omega*Derivative(phi(y), y)/y - 2*m*omega*(2*m*omega - I)*phi(y)/y**2 + Derivative(phi(y), (y, 2)))*exp(-2*I*m*omega/(y - 1))/(y**(2*I*m*omega)*(1 - y)**(2*I*m*omega)) + (3*y - 1)*(2*I*m*omega*phi(y)*exp(2*I*m*omega/(1 - y))/(y**(2*I*m*omega)*(1 - y)*(1 - y)**(2*I*m*omega)) + 2*I*m*omega*phi(y)*exp(2*I*m*omega/(1 - y))/(y**(2*I*m*omega)*(1 - y)**2*(1 - y)**(2*I*m*omega)) - 2*I*m*omega*phi(y)*exp(2*I*m*omega/(1 - y))/(y*y**(2*I*m*omega)*(1 - y)**(2*I*m*omega)) + exp(2*I*m*omega/(1 - y))*Derivative(phi(y), y)/(y**(2*I*m*omega)*(1 - y)**(2*I*m*omega)))/(y*(y - 1)) + (4*m**2*omega**2 - y*(y - 1)**2*(ell*(ell + 1) + 3*y - 3))*phi(y)*exp(2*I*m*omega/(1 - y))/(y**2*y**(2*I*m*omega)*(1 - y)

In [331]:
# Expand expression, collect terms according to order of phi
A = sp.collect(eq_yy_norm_bc.expand(), phi_y)
A

(4*I*m*omega/(y**2*y**(2*I*m*omega)*(1 - y)**(2*I*m*omega)*exp(2*I*m*omega/(y - 1)) - 2*y*y**(2*I*m*omega)*(1 - y)**(2*I*m*omega)*exp(2*I*m*omega/(y - 1)) + y**(2*I*m*omega)*(1 - y)**(2*I*m*omega)*exp(2*I*m*omega/(y - 1))) - 4*I*m*omega/(y*y**(2*I*m*omega)*(1 - y)**(2*I*m*omega)*exp(2*I*m*omega/(y - 1)) - y**(2*I*m*omega)*(1 - y)**(2*I*m*omega)*exp(2*I*m*omega/(y - 1))) - 4*I*m*omega*exp(-2*I*m*omega/(y - 1))/(y*y**(2*I*m*omega)*(1 - y)**(2*I*m*omega)) + 3*y*exp(2*I*m*omega/(1 - y))/(y**2*y**(2*I*m*omega)*(1 - y)**(2*I*m*omega) - y*y**(2*I*m*omega)*(1 - y)**(2*I*m*omega)) - exp(2*I*m*omega/(1 - y))/(y**2*y**(2*I*m*omega)*(1 - y)**(2*I*m*omega) - y*y**(2*I*m*omega)*(1 - y)**(2*I*m*omega)))*Derivative(phi(y), y) + (-ell**2*y**3*exp(2*I*m*omega/(1 - y))/(y**6*y**(2*I*m*omega)*(1 - y)**(2*I*m*omega) - 4*y**5*y**(2*I*m*omega)*(1 - y)**(2*I*m*omega) + 6*y**4*y**(2*I*m*omega)*(1 - y)**(2*I*m*omega) - 4*y**3*y**(2*I*m*omega)*(1 - y)**(2*I*m*omega) + y**2*y**(2*I*m*omega)*(1 - y)**(2*I*m*omega)

In [332]:
# Get coefficient of second derivative of phi
A_norm = A.coeff(sp.Derivative(sp.Derivative(phi_y, y), y))
A_norm

exp(-2*I*m*omega/(y - 1))/(y**(2*I*m*omega)*(1 - y)**(2*I*m*omega))

In [341]:
# Multiply expression by norm
A_normalized = A / A_norm
A_normalized_collected = sp.collect(A_normalized.simplify().expand(), phi_y)
A_normalized_collected

(y**3/(y**3 - 2*y**2 + y) - 2*y**2/(y**3 - 2*y**2 + y) + y/(y**3 - 2*y**2 + y))*Derivative(phi(y), (y, 2)) + (-8*I*m*omega*y**2/(y**3 - 2*y**2 + y) + 16*I*m*omega*y/(y**3 - 2*y**2 + y) - 4*I*m*omega/(y**3 - 2*y**2 + y) + 3*y**2/(y**3 - 2*y**2 + y) - 4*y/(y**3 - 2*y**2 + y) + 1/(y**3 - 2*y**2 + y))*Derivative(phi(y), y) + (-ell**2/(y**3 - 2*y**2 + y) - ell/(y**3 - 2*y**2 + y) - 16*m**2*omega**2*y/(y**3 - 2*y**2 + y) + 32*m**2*omega**2/(y**3 - 2*y**2 + y) - 8*I*m*omega*y/(y**3 - 2*y**2 + y) + 8*I*m*omega/(y**3 - 2*y**2 + y) - 3*y/(y**3 - 2*y**2 + y) + 3/(y**3 - 2*y**2 + y))*phi(y)

In [350]:
C_1 = A_normalized_collected.coeff(sp.Derivative(sp.Derivative(phi_y, y), y)).simplify()
C_1

1

In [357]:
C_2 = A_normalized_collected.coeff(sp.Derivative(phi_y, y)).simplify().factor()
C_2

-I*(8*m*omega*y**2 - 16*m*omega*y + 4*m*omega + 3*I*y**2 - 4*I*y + I)/(y*(y - 1)**2)

In [358]:
C_3 = A_normalized_collected.coeff(phi_y).simplify().factor()
C_3

-(ell**2 + ell + 16*m**2*omega**2*y - 32*m**2*omega**2 + 8*I*m*omega*y - 8*I*m*omega + 3*y - 3)/(y*(y - 1)**2)

Finally, write out the new equation, with the change of coordinates and the boundary conditions

In [353]:
eq_final = C_1 * sp.Derivative(sp.Derivative(phi_y, y), y) + C_2 * sp.Derivative(phi_y, y) + C_3 * phi_y
eq_final

Derivative(phi(y), (y, 2)) + (-8*I*m*omega*y**2 + 16*I*m*omega*y - 4*I*m*omega + 3*y**2 - 4*y + 1)*Derivative(phi(y), y)/(y*(y**2 - 2*y + 1)) + (-ell**2 - ell - 16*m**2*omega**2*y + 32*m**2*omega**2 - 8*I*m*omega*y + 8*I*m*omega - 3*y + 3)*phi(y)/(y*(y**2 - 2*y + 1))